OSIsoft Cloud Services REST API

requirements:<br>
Juptyer notebook or lab<br>
Python ocs_sample_library https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/library_samples/Python3<br>
config.ini configured for your OSIsoft Cloud Services environment https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/basic_samples/SDS/Python/SDSPy/Python3<br>

objective:<br>
Create metadata and tags for streams

In [2]:
# specify a unique prefix to create objects, suggestion use your nickname
example_prefix = 'SDSUoMPrefix.'

# run a fork-ed/edited version of the OSIsoft Python library to support Uoms
custom_ocs_library = True
   
# setup the environment, using statements defined in a python script file   
%run -i setup

In [3]:
# verify we have a connection to the tenant, the tenant name should be returned
ocsClient.tenant

'efe27258-f6d5-4ea6-a001-3e4a82777710'

In [6]:
# Specify the defintion for the type that will consist of: timestamp, index and channel

# Dictionary to describe information about the type, that will be used to define the type
thetype = {'id': f'{example_prefix}bearing_temperature', # unique identifier for the type
           'name': 'bearing_channel', 
           'description': 'bearing vibration signals'}

# property - timestamp
timestamp_property = SdsTypeProperty()
timestamp_property.Id = "timestamp"
timestamp_property.SdsType = SdsType.fromDictionary({"SdsTypeCode": SdsTypeCode.DateTime.value})
timestamp_property.IsKey = True

# property - temperature
temperature_property =  SdsTypeProperty()
temperature_property.Id = "temperature"
temperature_property.Uom = "degree Fahrenheit"
temperature_property.SdsType = SdsType.fromJson({"SdsTypeCode" : SdsTypeCode.Double.value })

# type defintion
bearing = SdsType()
bearing.SdsTypeCode = SdsTypeCode.Object
bearing.Id = thetype['id']
bearing.Name = thetype['name']
bearing.Description=thetype['description']
bearing.Properties = [timestamp_property,temperature_property]

# verbose! - not yet supported fully by OSIsoft library
ocsClient.acceptverbosity = True
# Create the type
bearing_type = ocsClient.Types.getOrCreateType(namespace_id, bearing)
print(json.dumps(json.loads(bearing_type.toJson()),indent=4))

{
    "SdsTypeCode": 1,
    "Properties": [
        {
            "Id": "timestamp",
            "Name": null,
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 16,
                "Id": null,
                "Name": null,
                "Description": null,
                "ExtrapolationMode": 0,
                "InterpolationMode": 0
            },
            "Value": null,
            "Order": 0,
            "IsKey": true,
            "InterpolationMode": null,
            "Uom": null
        },
        {
            "Id": "temperature",
            "Name": null,
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 14,
                "Id": null,
                "Name": null,
                "Description": null,
                "ExtrapolationMode": 0,
                "InterpolationMode": 0
            },
            "Value": null,
            "Order": 0,
            "IsKey": false,
            "Interpolatio

In [7]:
# Create a stream using the created type
stream_name = 'Asset3_bearing3'
new_stream = SdsStream(id=f'{example_prefix}{stream_name}',name=f'{stream_name} temperature1', description=f'{stream_name} temperature sensor1', typeId=f'{example_prefix}bearing_temperature')
stream = ocsClient.Streams.getOrCreateStream(namespace_id,new_stream)
print(f'id: {stream.Id}, name: {stream.Name}')

id: SDSUoMPrefix.Asset3_bearing3, name: Asset3_bearing3 temperature1


In [8]:
# Add stream events
ocsClient.Streams.updateValues(namespace_id,stream.Id,json.dumps(
[{'Timestamp': '2019-02-11T00:00:00Z', 'Temperature': 100},
 {'Timestamp': '2019-02-11T00:00:01Z', 'Temperature': 35.492},
 {'Timestamp': '2019-02-11T00:00:02Z', 'Temperature': 35.469},
 {'Timestamp': '2019-02-11T00:00:03Z', 'Temperature': 35.422},
 {'Timestamp': '2019-02-11T00:00:04Z', 'Temperature': 35.414},
 {'Timestamp': '2019-02-11T00:00:05Z', 'Temperature': 35.32},
 {'Timestamp': '2019-02-11T00:00:06Z', 'Temperature': 35.227},
 {'Timestamp': '2019-02-11T00:00:07Z', 'Temperature': 35.242},
 {'Timestamp': '2019-02-11T00:00:08Z', 'Temperature': 35.16},
 {'Timestamp': '2019-02-11T00:00:09Z', 'Temperature': 35.176}]))

# Request data converted to celsius

From the OCS Portal API Console request conversion to celsius using a POST request with the following JSON as the body:<br>
Note: substitute DerekETesting with your namespace id and if necessary update stream prefix

```
POST /Namespaces/DerekETesting/Streams/SDSUoMPrefix.Asset3_bearing3/Data/Transform/First

[
  {
    "SdsTypePropertyId" : "temperature",
    "Uom" : "degree Celsius" 
  }
]
```

# Create a stream using an override to specify celsius

From the OCS Portal API Console create a stream with an override to define the temperature property as celsius, instead of the type defined fahrenheit using the URI and POST body Note: substitute DerekETesting with your namespace id and if necesary update stream prefix
```
POST /Namespaces/DerekETesting/Streams/SDSUoMPrefix.Asset3_bearing4

{
    "TypeId": "SDSUoMPrefix.bearing_temperature",
    "Id": "SDSUoMPrefix.Asset3_bearing4",
    "Name": "bearing4 temperature1",
    "Description": "Bearing 4 temperature sensor1",
    "PropertyOverrides": [
  {
    "SdsTypePropertyId" : "temperature",
    "Uom" : "degree Celsius" 
  }
]
}
```

In [17]:
# write a value to the new stream
ocsClient.Streams.updateValues(namespace_id,f'{example_prefix}Asset3_bearing4',json.dumps([{'Timestamp': '2019-02-11T00:00:00Z', 'Temperature': 37.9}]))
# read the value
ocsClient.Streams.getFirstValue(namespace_id,f'{example_prefix}Asset3_bearing4',None)

{'timestamp': '2019-02-11T00:00:00Z', 'temperature': 37.9}

In [18]:
# Clean-up environment - delete type and streams with {example_prefix} prefix defined earlier
# Note: don't do this if you plan to run subsequent notebooks in this "group"

def cleanup(run=False):
        for stream in ocsClient.Streams.getStreams(namespace_id,f'{example_prefix}*'):
            print(f'stream id: {stream.Id}, stream name: {stream.Name}')
            if run:
                try:
                    ocsClient.Streams.deleteStream(namespace_id,stream.Id)
                except:
                    pass
        for type in ocsClient.Types.getTypes(namespace_id,query=f'{example_prefix}*'):
            print(f'type id: {type.Id}, type name: {type.Name}')
            if run:
                try:
                    ocsClient.Types.deleteType(namespace_id,type.Id)
                    ocsClient.Types.deleteType(namespace_id,type.TypeId)
                except:
                    pass
        
# cleanup - disabled, i.e: False and commented:
#cleanup(False)

stream id: SDSUoMPrefix.Asset3_bearing3, stream name: Asset3_bearing3 temperature1
stream id: SDSUoMPrefix.Asset3_bearing4, stream name: bearing4 temperature1
type id: SDSUoMPrefix.bearing_temperature, type name: bearing_channel
